# Алгоритм Витерби для каталанского корпуса

Импортируем библиотеки

In [1]:
import conllu # парсилка корпуса
from io import open
import pickle # для сохранения состояния обучения
import numpy as np

In [2]:
dir_in = "./UD_Catalan-AnCora/"
train_file = "ca_ancora-ud-train.conllu"
test_file = "ca_ancora-ud-test.conllu"

Открываем файл с тренировочным корпусом

In [3]:
data_train = conllu.parse_incr(open(dir_in+train_file,'r',encoding = "utf-8"))

Подсчитываем количество триграммов,биграммов и монограммов в корпусе,используя словари

In [4]:
c3 = {} # количество различных триграммов 
c2 = {} # //---//---// биграммов
c1 = {} # //---//---// монограммов
cT = {} # //---//---// пар слово-тег
for tokenlist in data_train:
    sentence = [["*","*"]] + [["*","*"]] + [[tokenlist[i]["lemma"], tokenlist[i]["upostag"]]
        for i in range(len(tokenlist)) ] + [["STOP","STOP"]]

    for i in range(0, len(sentence)-2):
        c3old = c3.get( (sentence[i][1], sentence[i+1][1], sentence[i+2][1]), 0)# если такого триграмма еще нет в словаре,вернуть 0
        c3.update( { (sentence[i][1], sentence[i+1][1], sentence[i+2][1]): c3old+1 } ) #если есть,увеличиваем количество на 1

    for i in range(0, len(sentence)-1):
        c2old = c2.get( (sentence[i][1], sentence[i+1][1]), 0)
        c2.update( { (sentence[i][1], sentence[i+1][1]): c2old+1 } )

    for i in range(0, len(sentence)):
        c1old = c1.get( (sentence[i][1]), 0)
        c1.update( { (sentence[i][1]): c1old+1 } )

    for i in range(0, len(sentence)):
        cTold = cT.get( (sentence[i][1], sentence[i][0]), 0)
        cT.update( { (sentence[i][1], sentence[i][0]): cTold+1 } )


Сохраняем данные в пикл, чтобы потом использовать, не обучая заново

In [5]:
print (len(c3))
print (len(c2))
print (len(c1))
print (len(cT))

# Saving:
with open('cs_catalan.pkl', 'wb') as f:
    pickle.dump([cT, c1, c2, c3], f)


2748
256
20
23534


Загружаем данные из файла обратно

In [6]:
# Getting back the cs:
with open('cs_catalan.pkl', 'rb') as f:
    cT, c1, c2, c3 = pickle.load(f)

In [7]:
tags = list(c1.keys())# список уникальных тегов
tags.remove('*')
tags.remove('STOP')
k = len(tags) # количество уникальных тегов

Вспомогательные функции из алгоритма Витерби

In [8]:
def q(s, u, v):
    eps = 1e-7
    return c3.get((u, v, s), eps) / c2.get((u, v), eps)


def e(x, s):
    eps = 1e-7
    return cT.get((s, x), eps) / c1.get((s), eps)


def K(n): # выдает возможные тэги в зависимости от положения слова в предложении (в начале предложения - *) 
    if n == -1 or n == 0:
        return ['*']
    else:
        return tags

print(K(10))


['ADP', 'AUX', 'ADV', 'DET', 'SCONJ', 'INTJ', 'SYM', 'PUNCT', 'PROPN', 'NUM', 'CCONJ', '_', 'NOUN', 'ADJ', 'X', 'PRON', 'VERB', 'PART']


Сам алгоритм Витерби в виде функции от токенизированного предложения ,возвращающей список тегов

In [9]:
def viterbi(sent):
    Pi = {(0, '*', '*'): 1}
    bp = {}
    n = len(sent) - 2
    y = [""] * (n + 1)

    for k in range(1, n + 1):
        xk = sent[k]
       
        for u in K(k-1):
            for v in K(k):
                w = K(k-2)
                v1 = map(lambda wi:
                            Pi.get((k-1, wi, u)) *
                            q(v, wi, u) *
                            e(xk, v), w)
                v1 = list(v1)
                PiNew = max(v1)
                bpNew = w[v1.index(PiNew)]
                Pi.update({(k, u, v): PiNew})
                bp.update({(k, u, v): bpNew})

    v2 = {}
    for u in K(n-1):
        for v in K(n):
            v2.update({(n, u, v): Pi.get((n, u, v)) * q("STOP", u, v)})

    v2max = max(list(v2.values()))
    for (n, u, v), val in v2.items():
        if val == v2max:
            (y[n-1], y[n]) = (u,v)

    for k in range(n-2, 0, -1):
        y[k] = bp.get((k + 2, y[k+1], y[k+2]))

    #print(y[1:])
    return y[1:]


Открываем тестовый корпус и прогоняем по нему алгоритм

In [10]:
data_test = conllu.parse_incr(open(dir_in+test_file,'r',encoding = "utf-8"))
test_tags = [] # оригинальные теги корпуса
predict_tags = [] # предсказанные алгоритмом

for tokenlist in data_test:
    sentence=["*"] + [tokenlist[i]["lemma"] for i in range(len(tokenlist)) ] + ["STOP"]
    test_tags.append([tokenlist[i]["upostag"] for i in range(len(tokenlist)) ])
    predict_tags.append(viterbi(sentence))

Считаем ошибку как отношение количества неправильно предсказанных тегов к общему числу тегов

In [11]:
err = 0.
for sent in range(len(test_tags)):
    err += np.mean(np.array(test_tags[sent]) != np.array(predict_tags[sent]))
print('Error = ', err/len(test_tags))   

Error =  0.05064406281395883
